# 2 Player FPSB Auction with symmetric unfirorm valuation distributions

## Imports

In [2]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [3]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [4]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [5]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [6]:
torch.cuda.is_available()

True

## Settings

In [7]:
## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
u_lo =0
u_hi =10


## Environment settings
#training batch size
batch_size = 2**16
input_length = 1

# model architecture
size_hidden_layer = 25

# optimization params
epoch = 9000
learning_rate = 5e-3
sigma = .1 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 50
plot_points = min(150, batch_size)
sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [8]:
# for evaluation
def optimal_bid(valuation):
    return valuation / 2

In [9]:
# Wrapper transforming a strategy to bidder, used by the optimizer
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [10]:
model = NeuralNetStrategy(input_length, size_hidden_layer = size_hidden_layer, requires_grad=False).cuda()

In [11]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = Environment(mechanism,
                  environment_agents = [], #dynamically built
                  max_env_size = 1, #
                  batch_size = batch_size,
                  n_players =2,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [16]:
def log_hyperparams(writer):
    writer.add_scalars('hyperparams',
                       {'batch_size': batch_size,
                        'size_hidden_layer': size_hidden_layer,
                        'learning_rate': learning_rate,
                        'sigma': sigma,
                        'n_perturbations': n_perturbations})

In [14]:
torch.cuda.empty_cache()
writer = SummaryWriter()
log_hyperparams(writer)
for e in range(epoch+1):
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = (b/v).mean()
        utility = -optimizer.step()
        writer.add_scalar('utility', utility, e)
        writer.add_scalar('share', share, e)
        #writer.add_graph(model, bidder.valuations)
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f}".format(e, share, utility))
                
        
        # subsample points and plot
        v = v.detach().cpu().numpy()[:plot_points]
        b= b.detach().cpu().numpy()[:plot_points]
        optimal = optimal_bid(v)
        
        fig = plt.figure()
        plt.plot(v,b, 'o', v, optimal, 'r-')
        
        writer.add_figure('function', fig, e)
        
        plt.show()
    else:
        utility = -optimizer.step()
        writer.add_scalar('utility', utility, e)    
        
torch.cuda.empty_cache()
writer.close()

Epoch 0: 	avg bid: 0.000000, 	utility: 0.000000
Epoch 50: 	avg bid: 0.082897, 	utility: 4.374060
Epoch 100: 	avg bid: 0.205631, 	utility: 3.879433
Epoch 150: 	avg bid: 0.308437, 	utility: 3.450031
Epoch 200: 	avg bid: 0.398155, 	utility: 3.089642
Epoch 250: 	avg bid: 0.482230, 	utility: 2.735321
Epoch 300: 	avg bid: 0.559276, 	utility: 2.425313
Epoch 350: 	avg bid: 0.613356, 	utility: 2.182617
Epoch 400: 	avg bid: 0.667853, 	utility: 1.965780
Epoch 450: 	avg bid: 0.711833, 	utility: 1.788049
Epoch 500: 	avg bid: 0.754673, 	utility: 1.603689
Epoch 550: 	avg bid: 0.786832, 	utility: 1.450887
Epoch 600: 	avg bid: 0.830644, 	utility: 1.291167
Epoch 650: 	avg bid: 0.857120, 	utility: 1.185754
Epoch 700: 	avg bid: 0.881649, 	utility: 1.053549
Epoch 750: 	avg bid: 0.894960, 	utility: 0.975950
Epoch 800: 	avg bid: 0.907660, 	utility: 0.903950
Epoch 850: 	avg bid: 0.920067, 	utility: 0.843992
Epoch 900: 	avg bid: 0.933576, 	utility: 0.787086
Epoch 950: 	avg bid: 0.942577, 	utility: 0.739020
Epo